In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import os
from tqdm import tqdm

In [ ]:

class MovieSearchEngine:
    def __init__(self, data_path="imdb_movies_success.csv"):
        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Файл {data_path} не найден.")
        
        self.df = self._load_and_clean_data(data_path)
        self.synonyms = self._get_synonyms()
        self.vectorizer = self._init_vectorizer()
        self._prepare_search_text()
        self.tfidf_matrix = self._train_model()
    
    def _load_and_clean_data(self, path):
        """Загрузка и глубокая очистка данных"""
        df = pd.read_csv(path)
        
        required_cols = {
            'page_url', 'image_url', 'movie_title', 'year',
            'description', 'director', 'actors', 'genre', 'rating'
        }
        missing = required_cols - set(df.columns)
        if missing:
            raise ValueError(f"Отсутствуют колонки: {missing}")

        text_cols = ['movie_title', 'description', 'director', 'actors', 'genre']
        for col in text_cols:
            df[col] = df[col].fillna('').astype(str)
        
        print(f"Успешно загружено {len(df)} фильмов")
        return df
    
    def _get_synonyms(self):
        """Словарь синонимов для улучшения поиска"""
        return {
            'космическая': ['космос', 'галактика', 'звезды', 'фантастика'],
            'комедия': ['смешной', 'юмор', 'веселый'],
            'драма': ['трагедия', 'серьезный'],
            'роман': ['любовь', 'романтический'],
            'робот': ['андроид', 'киборг']
        }
    
    def _init_vectorizer(self):
        """Инициализация векторизатора"""
        return TfidfVectorizer(
            max_features=100000,
            ngram_range=(1, 3),
            min_df=2,
            stop_words=['и','в','во','не','что','он','на','я','с','со','как','а']
        )
    
    def _prepare_search_text(self):
        """Подготовка текста для поиска"""
        print("Подготовка данных...")
        
        self.df['clean_title'] = self.df['movie_title'].str.lower().apply(self._clean_text)
        self.df['clean_genre'] = self.df['genre'].str.lower().str.replace(',', ' ')
        self.df['clean_actors'] = self.df['actors'].str.lower().str.replace(',', ' ')
        self.df['clean_director'] = self.df['director'].str.lower()
        self.df['clean_desc'] = self.df['description'].str.lower().apply(self._clean_text)
        
        self.df['search_text'] = (
            "TITLE: " + self.df['clean_title'] + " " +
            "GENRE: " + self.df['clean_genre'] + " " +
            "ACTORS: " + self.df['clean_actors'] + " " +
            "DIRECTOR: " + self.df['clean_director'] + " " +
            "DESC: " + self.df['clean_desc']
        )
    
    def _clean_text(self, text):
        """Очистка текста"""
        text = re.sub(r'[^\w\s]', ' ', text)  
        return ' '.join([w for w in text.split() if len(w) > 2]) 
    
    def _train_model(self):
        """Обучение модели"""
        print("Обучение модели...")
        valid_texts = self.df[self.df['search_text'].str.strip() != '']['search_text']
        if len(valid_texts) == 0:
            raise ValueError("Нет текста для обучения")
        return self.vectorizer.fit_transform(valid_texts)
    
    def _expand_query(self, query):
        """Расширение запроса синонимами"""
        query = query.lower()
        expanded = [query]
        for term, syns in self.synonyms.items():
            if term in query:
                for syn in syns:
                    expanded.append(query.replace(term, syn))
        return ' '.join(list(set(expanded)))
    
    def search(self, query, top_n=5):
        """Поиск фильмов"""
        try:
            expanded_query = self._expand_query(query)
            query_vec = self.vectorizer.transform([expanded_query])
            
            similarities = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
            top_indices = np.argsort(similarities)[-top_n:][::-1]
            
            results = self.df.iloc[top_indices].copy()
            results['similarity'] = similarities[top_indices]
            
            results = results[results['similarity'] > 0.1]
            
            if not results.empty:
                return self._format_results(results)
            return pd.DataFrame()
        
        except Exception as e:
            print(f"Ошибка поиска: {e}")
            return pd.DataFrame()
    
    def _format_results(self, results):
        """Форматирование результатов"""
        return results[[
            'movie_title', 'year', 'genre', 'director',
            'actors', 'description', 'similarity'
        ]].rename(columns={
            'movie_title': 'Название',
            'year': 'Год',
            'genre': 'Жанр',
            'director': 'Режиссер',
            'actors': 'Актеры',
            'description': 'Описание',
            'similarity': 'Сходство'
        })

def main():
    try:
        data_file = "imdb_movies_success.csv"
        
        print("Загрузка системы поиска фильмов...")
        engine = MovieSearchEngine(data_file)
        
        queries = [
            "военная драма",
            "романтическая история"
        ]
        
        for query in queries:
            print(f"\nРезультаты для '{query}':")
            results = engine.search(query)
            
            if not results.empty:
                print(results.to_string(index=False, float_format="%.3f"))
            else:
                print("Ничего не найдено")
    
    except Exception as e:
        print(f"Ошибка: {e}")

if __name__ == "__main__":
    main()

Загрузка системы поиска фильмов...
Успешно загружено 5330 фильмов
Подготовка данных...
Обучение модели...

Результаты для 'военная драма':
              Название  Год                               Жанр                                   Режиссер                                                                                                                                                      Актеры                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        